In [ ]:
### THE CODE BELOW WORKS ONLY IN THE GCP GOOGLE FUNCTIONS; MUST COPY, PASTE AND RUN THERE
### This code only updates the data already in the SQL Database (i.e. does not serve as basic information and does not create new tables)
### The "Google Cloud Instance" IN MySQLWorkbench MIGHT need to be active to allow the information transfer
### This complete file NEEDS to be run before adding updated information to SQL database tables in the future.

# requirements:
# functions_framework  CHANGE; should be with -
# SQLAlchemy
# PyMySQL
# pandas
# requests

import functions_framework
import pandas as pd
import sqlalchemy
import requests

@functions_framework.http
def insert_flights_parameters(request):
    connection_string = connection()
    insert_flights_data_in_SQL(connection_string)
    return "Data successfully added - all Ok"

def connection():
    schema = "gans_data_pipeline"  # update "gans_data_pipeline" to your actual MySql database (schema) name
    host = "127.0.0.1"  # update with your own cloud SQL instance public IP address
    user = "root"  # update with your MySql username
    password = "password" # update "password" with your actual MySql password
    port = 3306  # update with your own default MySQL port, if different
    connection_string = f"mysql+pymysql://{user}:{password}@{host}:{port}/{schema}"
    return connection_string

def insert_flights_data_in_SQL (connection_string):
  utc_now = pd.Timestamp.utcnow()
  tomorrow = utc_now + pd.Timedelta(hours=24)
  tomorrow_plus_12 = tomorrow + pd.Timedelta(hours=12)
  airports_from_sql = pd.read_sql("airports", con=connection_string)
  airport_icao = airports_from_sql['icao_code'].to_list()
  airlines = []
  flight_numbers = []
  arrival_times = []
  arrival_terminals = []
  all_arrivals = []
  icao_code = []

  for codes in airport_icao:
      url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{codes}/{tomorrow}/{tomorrow_plus_12}"
      querystring = {"direction":"Arrival","withPrivate":"true"}
      headers = {
        "x-rapidapi-key": "687292277emsh6620811a3972b04p1a4ee9jsn8c02f9bc139b", #f"{api_key}"
        "x-rapidapi-host": "aerodatabox.p.rapidapi.com"
      }

      try:
          arrivals_response = requests.get(url, headers=headers, params=querystring)

          if arrivals_response.status_code == 200:
              arrivals_response_json = arrivals_response.json()

              for i in arrivals_response_json['arrivals']:
                  airlines.append(i['airline'].get('name',None))#airline
                  flight_numbers.append(i.get('number', None))#flight numbers
                  arrival_times.append(i['movement']['scheduledTime'].get('utc', None)) #all arriving flights times
                  arrival_terminals.append(i['movement'].get('terminal', None)) #terminals
                  icao_code.append(codes)  # append the current ICAO code
            
              for codes in airport_icao:
                  icao_code.append(codes)

      except requests.exceptions.RequestException as e:
          print(f"Error fetching data for airport code {codes}: {e}")

  if airlines:
      flights_dataframe = pd.DataFrame(list(zip(icao_code, airlines,flight_numbers, arrival_times, arrival_terminals)),
                                  columns = ['icao_code','airlines', 'flight_numbers', 'arrival_times', 'arrival_terminals'])

      airports_from_sql = pd.read_sql("airports", con=connection_string)
      
      merged_flights_df = flights_dataframe.merge(airports_from_sql,
                                   on = "icao_code",
                                   how="left")
      
      flights_df = merged_flights_df.drop(columns=["icao_code","iata_code", "airport_name","city_name", "city_id"])
      
      flights_df['arrival_times']=pd.to_datetime(flights_df['arrival_times'])
      
      
      flights_df.to_sql('flights',
                   if_exists='append',
                   con=connection_string,
                   index=False)

  else:
      print("No arrivals data collected.")
